In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Loading targets into stk from file.
stk_object.set_sim_time(days=0,hours=1)
stk_object.Target_Loader("../../Input_Files/Target_Packages/Targets_15.txt")

In [3]:
n_pop = 2
n_gen = 1
n_sats = 4
opt = Optimizer(stk_object,n_pop,n_gen,n_sats)

In [11]:
print("Beginning Optimization")
with open(f"../../Pop_Over_Gen/{n_sats}.csv","w") as f:
    hof = opt.run(read=True,enable_print=True,file=f)
opt.Load_Individual(hof[0])


-- Generation 1 --
        avg       std       min       max
0  7.644033  0.360082  7.283951  8.004115
1  5.412078  0.708552  4.703525  6.120630
2  0.039806  0.000006  0.039799  0.039812
3  0.007066  0.000064  0.007002  0.007130


In [48]:
hof[0]

[584, 115, 152, 19, 1]

In [43]:
df = pd.read_csv(f"../../Pop_Over_Gen/{n_sats}.csv")
if (df[(df['Alt'] == hof[0][0]) & (df['Inc'] == hof[0][1]) & (df['Initial_Raan'] == hof[0][2]) & (df['Delta_Raan'] == hof[0][3]) & (df['Num_Planes'] == hof[0][4])].iloc[0]['Avg_Percentage'] == 100):
    print(True)

TypeError: unhashable type: 'Satellite'

In [25]:
pd.read_csv(f"../../Input_Files/Constellations/{n_sats}.txt").to_csv(f"../../Input_Files/Satellites_File_{n_sats}.txt",index=False)

print("Beginning Optimization")
with open(f"../../Pop_Over_Gen/{n_sats}.csv","w") as f:
    hof = opt.run(read=True,enable_print=True,file=f)

opt.Load_Individual(hof[0])

stk_object.set_sim_time(days=0, hours=1)
stk_object.dt = 60

message = [[f'{n_sats} Sats', 'Avg Unplanned %', 'Max Unplanned Time']]
for n_targets in [15,34,65,82]: #,109,186,494]:
    stk_object.Target_Loader(f"../../Input_Files/Target_Packages/Targets_{n_targets}.txt")
    stk_object.Generate_Pre_Planning_Data()
    data_comparison = {}
    data_comparison["Unplanned (%)"] = [np.count_nonzero(stk_object.target_bins[tar_num])/324*100 for tar_num in range(len(stk_object.targets))]
    data_comparison["Unplanned (Time)"] = [np.max(stk_object.target_times[tar_num])/86400 for tar_num in range(len(stk_object.targets))]
    pd.DataFrame(data_comparison).to_csv(f"../../Optimization_Results/{n_sats}_{n_targets}.csv",index=False)
    data_comparison = pd.DataFrame(data_comparison)

    message.append([data_comparison["Unplanned (%)"].count(), data_comparison["Unplanned (%)"].mean(), data_comparison["Unplanned (Time)"].max()])




-- Generation 1 --
        avg       std       min       max
0  2.664860  1.385125  1.279735  4.049985
1  4.288314  1.440278  2.848037  5.728592
2  0.041006  0.000261  0.040745  0.041267
3  0.004133  0.000531  0.003602  0.004664
- Computing_Access [==========] 60/60 [100%] in 0.2s (415.56/s)                 
- Getting_AzEl [==========] 60/60 [100%] in 0.4s (147.74/s)                     
- Sorting_Data [==========] 4860/4860 [100%] in 0.0s (113594.64/s)              
- Computing_Access [==========] 136/136 [100%] in 0.4s (363.27/s)               
- Getting_AzEl [==========] 136/136 [100%] in 0.9s (151.95/s)                   ▄▂▂ 102/136 [75%] in 1s (~0s, 179.6/s) 
- Sorting_Data [==========] 11016/11016 [100%] in 0.1s (292322.23/s)            
- Computing_Access [==========] 260/260 [100%] in 0.9s (279.51/s)               
- Getting_AzEl [==========] 260/260 [100%] in 2.5s (104.96/s)                   
- Sorting_Data [==========] 21060/21060 [100%] in 0.1s (365854.29/s)            
- C

In [26]:
df = pd.DataFrame(message, columns=message[0]).drop(0)
print(f'{df}')

  4 Sats Avg Unplanned % Max Unplanned Time
1     15        2.757202           0.041667
2     34         3.75817           0.041667
3     65        4.392213           0.041667
4     82        4.049985           0.041667


In [ ]:
send_message_to_discord(message='help',channel_id=1293765673649963038)